In [ ]:
#11.7.1.1. 类别预测层
#在下面，我们定义了这样一个类别预测层，通过参数 num_anchors 和 num_classes 分别指定了  a  和  q  。 
#该图层使用填充为 1 的  3×3  的卷积层。此卷积层的输入和输出的宽度和高度保持不变。
!pip install d2l
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


def cls_predictor(num_inputs, num_anchors, num_classes):
    return nn.Conv2d(num_inputs, num_anchors * (num_classes + 1),
                     kernel_size=3, padding=1)

In [ ]:
#11.7.1.2. 边界框预测层
#边界框预测层的设计与类别预测层的设计类似。 唯一不同的是，这里需要为每个锚框预测4个偏移量，而不是  q+1  个类别。
def bbox_predictor(num_inputs, num_anchors):
    return nn.Conv2d(num_inputs, num_anchors * 4, kernel_size=3, padding=1)

In [ ]:
#11.7.1.3. 连接多尺度的预测
def forward(x, block):
    return block(x)

Y1 = forward(torch.zeros((2, 8, 20, 20)), cls_predictor(8, 5, 10))
Y2 = forward(torch.zeros((2, 16, 10, 10)), cls_predictor(16, 3, 10))
Y1.shape, Y2.shape

In [ ]:
#通道维包含中心相同的锚框的预测结果。我们首先将通道维移到最后一维。 因为不同尺度下批量大小仍保持不变，
#我们可以将预测结果转成二维的（批量大小，高  ×  宽  ×  通道数）的格式，以方便之后在维度  1  上的连结。
def flatten_pred(pred):
    return torch.flatten(pred.permute(0, 2, 3, 1), start_dim=1)

def concat_preds(preds):
    return torch.cat([flatten_pred(p) for p in preds], dim=1)

In [ ]:
#这样一来，尽管 Y1 和 Y2 在通道数、高度和宽度方面具有不同的大小，
#我们仍然可以在同一个小批量的两个不同尺度上连接这两个预测输出。
concat_preds([Y1, Y2]).shape

In [ ]:
#11.7.1.4. 高和宽减半块
def down_sample_blk(in_channels, out_channels):
    blk = []
    for _ in range(2):
        blk.append(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        blk.append(nn.BatchNorm2d(out_channels))
        blk.append(nn.ReLU())
        in_channels = out_channels
    blk.append(nn.MaxPool2d(2))
    return nn.Sequential(*blk)

In [ ]:
#在以下示例中，我们构建的高和宽减半块会更改输入通道的数量，并将输入特征图的高度和宽度减半。
forward(torch.zeros((2, 3, 20, 20)), down_sample_blk(3, 10)).shape